In [1]:
import pickle
import pandas as pd
from datetime import timedelta
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
if os.path.basename(os.getcwd())=="Belfort":
    os.chdir('..')
    os.chdir('..') ## to work at project root  like in any IDE

InputFolder='SujetsDAnalyses/Belfort/'
from functions.f_consumptionModels import *

# 1. Import data

In [2]:
Temp_df=pd.read_csv('Data/input/Conso_model/Temp_FR_2017_2022.csv',sep=';',decimal='.',parse_dates=['Date']).set_index(["Date"])

index2019=(Temp_df.index.to_series().dt.minute==0)&(Temp_df.index.to_series().dt.year==2019)
Temp_2019_df=Temp_df[index2019].reset_index().set_index("Date").sort_index()
Temp_2019_df= CleanCETIndex(Temp_2019_df)# Traitement heure d'été et heure d'hiver

In [3]:
Conso_2019_df=pd.read_csv('Data/input/Conso_model/areaConsumption2019_FR.csv',sep=';',parse_dates=['Date']).set_index(["Date"])

In [94]:
year=2060
nuc="nuclear_plus"
type_result="multinode_"#"" if singlenode
bati_hyp="SNBC"
reindus="reindus"
with open(InputFolder+"Results_"+type_result+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus+".pickle", 'rb') as f:
    Variables=pickle.load(f)

In [5]:
print(Variables)

{'power_Dvar':        AREAS                TIME    TECHNOLOGIES   power_Dvar
0         GB 2019-02-25 07:00:00         OldNuke  1188.000000
1         GB 2019-02-25 07:00:00    WindOffShore  3718.938825
2         GB 2019-02-25 07:00:00         Biomass  4528.000000
3         GB 2019-02-25 07:00:00     WindOnShore  4899.552606
4         GB 2019-02-25 07:00:00             CCG     0.000000
...      ...                 ...             ...          ...
858475    CH 2019-05-26 10:00:00            Coal     0.000000
858476    CH 2019-05-26 10:00:00             TAC     0.000000
858477    CH 2019-05-26 10:00:00      Geothermal     0.000000
858478    CH 2019-05-26 10:00:00  HydroReservoir     0.000000
858479    CH 2019-05-26 10:00:00         NewNuke     0.000000

[858480 rows x 4 columns], 'exchange_Pvar':        AREAS AREAS                TIME  exchange_Pvar
0         GB    GB 2019-02-25 07:00:00            0.0
1         GB    GB 2019-05-17 00:00:00            0.0
2         GB    GB 2019-02-10 08:0

In [6]:
StockTechnos_df=pd.read_csv("Data/input/Prod_model/Prod_Europe/Stock_technos_Europe_"+str(year)+".csv",
                            delimiter=";",decimal=",").set_index(['AREAS','STOCK_TECHNO'])

# 2. Tools

## 2.1. Capacity

In [7]:
def prod_capa(area):
    df=Variables['capacity_Dvar']
    return df[df.AREAS==area].set_index(['AREAS','TECHNOLOGIES']).sort_index()[['capacity_Dvar']]

def stock_capa(area):
    df1=Variables['Pmax_in_Dvar'].set_index(['AREAS','STOCK_TECHNO'])
    df2=Variables['Pmax_out_Dvar'].set_index(['AREAS','STOCK_TECHNO'])
    df3=Variables['Cmax_Pvar'].set_index(['AREAS','STOCK_TECHNO'])
    df1['Pmax_out_Dvar']=df2['Pmax_out_Dvar']
    df1['Cmax_Pvar']=df3['Cmax_Pvar']
    df1['hour_stock']=df1['Cmax_Pvar']/(df1['Pmax_in_Dvar']*StockTechnos_df['efficiency_in'])
    df1=df1.fillna(0)
    return df1.loc[(area,slice(None)),['Pmax_in_Dvar','Pmax_out_Dvar','Cmax_Pvar','hour_stock']].sort_index()

## 2.2. Energy

In [8]:
def energy(area):
    df=Variables['power_Dvar']
    df=df[df.AREAS==area]
    df=df.groupby(['AREAS','TECHNOLOGIES']).sum()
    df_capa=prod_capa(area)
    df['Load_factor']=df['power_Dvar']/(8760*df_capa['capacity_Dvar'])
    E=df['power_Dvar'].sum()
    df['Part']=df['power_Dvar']/E
    df=df.fillna(0)
    return df

## 2.3. Plots

In [9]:
def filter_variable(name,area,month_ini,day_ini,month_end,day_end):
    df=Variables[name]
    if month_ini==month_end:
        df=df[(df.AREAS==area)&(df.TIME.dt.month>=month_ini)&(df.TIME.dt.day>=day_ini)\
         &(df.TIME.dt.month<=month_end)&(df.TIME.dt.day<=day_end)]
    else:
        df=df[(df.AREAS==area)&(((df.TIME.dt.month==month_ini)&(df.TIME.dt.day>=day_ini))\
             |((df.TIME.dt.month<month_end)&(df.TIME.dt.month>month_ini))\
             |((df.TIME.dt.month==month_end)&(df.TIME.dt.day<=day_end)))]
    return df

def plot_prod(area,month_ini,day_ini,month_end,day_end,title="",year=2050):
    if area=='FR':
        df_prod=filter_variable('power_Dvar',area,month_ini,day_ini,month_end,day_end)
        df_sto_in=filter_variable('storageIn_Pvar',area,month_ini,day_ini,month_end,day_end)
        df_sto_out=filter_variable('storageOut_Pvar',area,month_ini,day_ini,month_end,day_end)
        
        df_exchange=Variables['exchange_Pvar']
        df_exchange.columns=['AREAS_from','AREAS','TIME','exchange_Pvar']
        df_exchange=df_exchange[(df_exchange.AREAS==area)&(df_exchange.TIME.dt.month>=month_ini)\
                                &(df_exchange.TIME.dt.day>=day_ini)\
         &(df_exchange.TIME.dt.month<=month_end)&(df_exchange.TIME.dt.day<=day_end)]
        
        
        df_prod=df_prod.pivot(index=['AREAS','TIME'], columns='TECHNOLOGIES', values='power_Dvar')
        df_sto_in=df_sto_in.pivot(index=['AREAS','TIME'], columns='STOCK_TECHNO', values='storageIn_Pvar')
        df_sto_out=df_sto_out.pivot(index=['AREAS','TIME'], columns='STOCK_TECHNO', values='storageOut_Pvar')
        
        df_exchange=df_exchange.pivot(index=['AREAS','TIME'], columns='AREAS_from', values='exchange_Pvar')
        
        df=pd.DataFrame()
        df['Nucléaire']=df_prod['OldNuke']+df_prod['NewNuke']
        df['Bioénergies et autres ENR']=df_prod['Biomass']+df_prod['Geothermal']
        df['Éolien']=df_prod['WindOffShore']+df_prod['WindOnShore']
        df['Solaire']=df_prod['Solar']
        df['Hydraulique']=df_prod['HydroRiver']+df_prod['NewHydroRiver']+df_prod['HydroReservoir']
        df['Gaz']=df_prod['CCG']+df_prod['TAC']
        df['Charbon']=df_prod['Coal']
        
        df['Injections batteries']=df_sto_out['Battery']
        df['Turbinage STEP']=df_sto_out['HydroStorage']
        df['Hydrogen-to-power']=df_sto_out['PowertoH2toPower']
        
        df['Imports']=sum(df_exchange[col] for col in df_exchange.columns)
        
        df['Soutirage batteries']=-df_sto_in['Battery']
        df['Pompage STEP']=-df_sto_in['HydroStorage']
        df['Power-to-hydrogen']=-df_sto_in['PowertoH2toPower']
        
        df['Exports']=df['Imports'].apply(lambda x: x if x<0 else 0)
        df['Imports']=df['Imports'].apply(lambda x: x if x>=0 else 0)
        
        d_colors={'Nucléaire':'yellow','Bioénergies et autres ENR':'darkgreen',
                 'Éolien':'lightskyblue','Solaire':'orange','Hydraulique':'blue',
                 'Gaz':'red','Charbon':'saddlebrown','Injections batteries':'magenta',
                 'Turbinage STEP':'navy','Hydrogen-to-power':'purple','Imports':'gray',
                 'Soutirage batteries':'magenta','Pompage STEP':'navy',
                 'Power-to-hydrogen':'purple','Exports':'gray'}
    
        
        #df=df.reset_index().melt(id_vars=['AREAS','TIME'], 
                                 #var_name="Moyens de production", value_name="Puissance (MW)")
        #df["Group"]=df["Moyens de production"].apply(lambda x: "-" if x in ['Soutirage batteries',
                                                                           #'Pompage STEP','Power-to-hydrogen',
                                                                           #'Exports'] else "+")
        df.reset_index(inplace=True)
        df['TIME']=df['TIME'].apply(lambda x: x.replace(year=year))
        
        fig=go.Figure()
        for col in df.columns:
            if col not in ['AREAS','TIME','Soutirage batteries','Pompage STEP','Power-to-hydrogen','Exports']:
                fig.add_trace(go.Scatter(x= df['TIME'], y = df[col]/1000,
                                     name = col,
                                     line=dict(width=0, color=d_colors[col]),
                                     stackgroup = '+'))
            elif col in ['Soutirage batteries','Pompage STEP','Power-to-hydrogen','Exports']:
                fig.add_trace(go.Scatter(x= df['TIME'], y = df[col]/1000,
                                     name = col,
                                     line=dict(width=0, color=d_colors[col]),
                                     stackgroup = '-'))
                
        fig.update_layout(title =title,
                           title_font_size = 20,width = 1000, height = 500)
        fig.update_yaxes(title_text = 'Puissance (GW)')
        fig.show()
        fig.write_image(InputFolder+"Graphs/Prod_"+type_result+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus\
                         +"_"+str(day_ini)+"_"+str(month_ini)+"_"+str(day_end)+"_"+str(month_end)+".png")
        
        #fig = px.area(df, x="TIME", y="Puissance (MW)", 
            #color="Moyens de production",category_orders={"Moyens de production":["Exports","Power-to-hydrogen",
            #"Pompage STEP","Soutirage batteries","Nucléaire","Solaire","Éolien","Hydraulique",
            #"Bioénergies et autres ENR","Gaz","Charbon","Injections batteries","Turbinage STEP",
            #"Hydrogen-to-power","Imports"]},
             #color_discrete_sequence=["yellow","yellow","orange"],
             #labels={"TIME":""},
             #title="",line_group='Group')
        #fig.show()
    else:
        print("Not implemented")

In [98]:
# Ça n'a rien à faire ici, mais c'est très joli !
plot_prod('FR',1,14,1,20)

## 2.4. Thermosensitivity

In [68]:
def moyenne_dict(d):
    return 1/max(1,len(d))*sum(d[x] for x in d)
    
def Thermosensitivity(area):
    Data_2019_df=Conso_2019_df.copy()
    Data_2019_df['Temperature']=Temp_2019_df['Temperature']
    Data_2019_df=Data_2019_df.rename(columns={'Consommation':'Consumption'})
    _,Th_w_2019,Th_s_2019=Decomposeconso2(Data_2019_df,T0=15,T1=20,TemperatureName='Temperature',
                    ConsumptionName='Consumption',TimeName='Date')
    
    Data_df=Variables['total_consumption_Pvar']
    Data_df=Data_df[Data_df.AREAS==area][['TIME','total_consumption_Pvar']].rename(columns={'TIME':'Date','total_consumption_Pvar':'Consumption'}).set_index('Date')
    Data_df['Temperature']=Temp_2019_df['Temperature']+(year-2019)*0.01
    _,Th_w,Th_s=Decomposeconso2(Data_df,T0=15,T1=20,TemperatureName='Temperature',
                    ConsumptionName='Consumption',TimeName='Date')
    
    
    print('Thermosensitivity winter 2019 ={} GW/°C'.format(moyenne_dict(Th_w_2019)))
    print('Thermosensitivity winter '+str(year)+' ={} GW/°C'.format(moyenne_dict(Th_w)))
    print('Thermosensitivity summer 2019 ={} GW/°C'.format(moyenne_dict(Th_s_2019)))
    print('Thermosensitivity summer '+str(year)+' ={} GW/°C'.format(moyenne_dict(Th_s)))
    
    x=list(range(0,24))
    y_w_2019=[-Th_w_2019[x] for x in Th_w_2019]
    y_w=[-Th_w[x] for x in Th_w]
    
    fig_w=go.Figure()
    fig_w.add_trace(go.Scatter(x=x,y=y_w_2019,mode="lines",
        line=go.scatter.Line(color="gray"),name="2019"))
    fig_w.add_trace(go.Scatter(x=x,y=y_w,mode="lines",
        line=go.scatter.Line(color="blue"),name=str(year)))
    fig_w.update_layout(title='En hiver',width = 1000, height = 500)
    fig_w.update_yaxes(title_text = 'Thermosensibilité (GW/°C)')
    fig_w.update_xaxes(title_text = 'Horaire')
    fig_w.show()
    
    y_s_2019=[Th_s_2019[x] for x in Th_s_2019]
    y_s=[Th_s[x] for x in Th_s]
    
    fig_s=go.Figure()
    fig_s.add_trace(go.Scatter(x=x,y=y_s_2019,mode="lines",
        line=go.scatter.Line(color="gray"),name="2019"))
    fig_s.add_trace(go.Scatter(x=x,y=y_s,mode="lines",
        line=go.scatter.Line(color="blue"),name=str(year)))
    fig_s.update_layout(title='En été',width = 1000, height = 500)
    fig_s.update_yaxes(title_text = 'Thermosensibilité (GW/°C)')
    fig_s.update_xaxes(title_text = 'Horaire')
    fig_s.show()
    
    fig_w.write_image(InputFolder+"Graphs/Thermo_winter_"+type_result+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus\
                         +".png")
    fig_s.write_image(InputFolder+"Graphs/Thermo_summer_"+type_result+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus\
                         +".png")
        
    
    


In [69]:
Thermosensitivity('FR')

Thermosensitivity winter 2019 =-2545.375218637865 GW/°C
Thermosensitivity winter 2050 =-2084.2149232749716 GW/°C
Thermosensitivity summer 2019 =900.4022823873376 GW/°C
Thermosensitivity summer 2050 =1969.3922731950693 GW/°C


## 2.5. Peak Load

In [77]:
def peak_load(area):
    Conso_df=Variables['total_consumption_Pvar']
    Conso_df=Conso_df[Conso_df.AREAS==area].set_index('TIME')
    H2_df=Variables['storageIn_Pvar']
    H2_df=H2_df[(H2_df.AREAS==area)&(H2_df.STOCK_TECHNO=='PowertoH2toPower')].set_index('TIME')
    Conso_df['total_consumption_Pvar']+=H2_df['storageIn_Pvar']
    return Conso_df[Conso_df['total_consumption_Pvar']==Conso_df['total_consumption_Pvar'].max()]

In [95]:
peak_load('FR')

,AREAS,total_consumption_Pvar
TIME,,
2019-01-28 13:00:00,FR,157818.231644


# 3. Capacity and production 

In [11]:
# Production means capacity (MW)
prod_capa('FR')

capacity_Dvar
AREAS TECHNOLOGIES                 
FR    Biomass             3000.0000
      CCG                    0.0000
      Coal                   0.0000
      Geothermal             0.0000
      HydroReservoir      8785.0000
      HydroRiver         11030.0000
      NewHydroRiver       1200.0000
      NewNuke            26730.0000
      OldNuke            13870.0000
      Solar             152960.2812
      TAC                    0.0000
      WindOffShore       49000.0000
      WindOnShore        37000.0000
      curtailment            0.0000

In [12]:
# Storage capacity
stock_capa('FR')

Pmax_in_Dvar  Pmax_out_Dvar     Cmax_Pvar  hour_stock
AREAS STOCK_TECHNO                                                           
FR    Battery            4467.740025    4467.740025  4.702884e+03    1.108033
      HydroStorage      10899.000000   10899.000000  1.961820e+05   20.000000
      PowertoH2toPower  17980.657471       0.000000  2.064981e+06  164.063651

In [13]:
energy('FR')

power_Dvar  Load_factor      Part
AREAS TECHNOLOGIES                                       
FR    Biomass         2.246106e+07     0.854683  0.027869
      CCG             0.000000e+00     0.000000  0.000000
      Coal            0.000000e+00     0.000000  0.000000
      Geothermal      0.000000e+00     0.000000  0.000000
      HydroReservoir  1.437226e+07     0.186758  0.017833
      HydroRiver      4.932150e+07     0.510454  0.061197
      NewHydroRiver   5.025825e+06     0.478104  0.006236
      NewNuke         1.908194e+08     0.814928  0.236764
      OldNuke         8.684522e+07     0.714768  0.107756
      Solar           2.104520e+08     0.157062  0.261124
      TAC             0.000000e+00     0.000000  0.000000
      WindOffShore    1.443606e+08     0.336317  0.179119
      WindOnShore     8.228838e+07     0.253882  0.102102
      curtailment     0.000000e+00     0.000000  0.000000

In [14]:
Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()

148910.4360652248

In [15]:
Pow_df=Variables['power_Dvar']
Pow_df=Pow_df[Pow_df.AREAS=='FR']
Pow_df['power_Dvar'].sum()/1E6

805.9462186422799

In [16]:
Conso_df=Variables['total_consumption_Pvar']
Conso_df=Conso_df[Conso_df.AREAS=='FR']
Conso_df['total_consumption_Pvar'].sum()/1E6

682.7512848930962

In [17]:
P2H2_df=Variables['storageIn_Pvar']
P2H2_df=P2H2_df[(P2H2_df.STOCK_TECHNO=='PowertoH2toPower')&(P2H2_df.AREAS=='FR')]
P2H2_df['storageIn_Pvar'].sum()/1E6

96.23518718954679

In [18]:
H2P_df=Variables['storageOut_Pvar']
H2P_df=H2P_df[(H2P_df.STOCK_TECHNO=='PowertoH2toPower')&(P2H2_df.AREAS=='FR')]
H2P_df['storageOut_Pvar'].sum()/1E6

0.0

# 4. Thermosensitivity

In [25]:
Data_2019_df=Conso_2019_df.copy()
Data_2019_df['Temperature']=Temp_2019_df['Temperature']
Data_2019_df=Data_2019_df.rename(columns={'Consommation':'Consumption'})
df,Th_w_2019,Th_s_2019=Decomposeconso2(Data_2019_df,T0=15,T1=20,TemperatureName='Temperature',
                    ConsumptionName='Consumption',TimeName='Date')
print(Th_w_2019)
print(Th_s_2019)

{0: -2381.9058772979797, 1: -2424.5466386259327, 2: -2469.388890814654, 3: -2482.9632570131944, 4: -2453.023770021789, 5: -2458.74922624812, 6: -2543.76006163136, 7: -2688.458969317609, 8: -2752.1101850298837, 9: -2620.346133962741, 10: -2558.714198694341, 11: -2574.3563450247552, 12: -2572.500053541472, 13: -2593.006564906592, 14: -2559.0727698661403, 15: -2501.756367465761, 16: -2548.83760242875, 17: -2622.0329162710527, 18: -2972.4764624469904, 19: -2977.9021902218165, 20: -2627.620932539371, 21: -2485.586218296157, 22: -2350.5583410637505, 23: -1869.3312745785418}
{0: 831.8360599016585, 1: 704.0659955292981, 2: 719.5801776142312, 3: 678.8690214059357, 4: 597.1055912875349, 5: 556.2094420678245, 6: 579.844650605531, 7: 740.2511054876878, 8: 941.6859551009035, 9: 1035.1324856658555, 10: 982.0738311481781, 11: 1232.174977190789, 12: 1079.3871621908163, 13: 917.3709409829777, 14: 957.1569548756152, 15: 940.6718162112325, 16: 967.570546454397, 17: 1111.036743261954, 18: 1130.71996491032

In [27]:
Data_df=Variables['total_consumption_Pvar']
Data_df=Data_df[Data_df.AREAS=='FR'][['TIME','total_consumption_Pvar']].rename(columns={'TIME':'Date','total_consumption_Pvar':'Consumption'}).set_index('Date')
Data_df['Temperature']=Temp_2019_df['Temperature']
df,Th_w,Th_s=Decomposeconso2(Data_df,T0=15,T1=20,TemperatureName='Temperature',
                    ConsumptionName='Consumption',TimeName='Date')
print(Th_w)
print(Th_s)

{0: -2578.665971710148, 1: -2795.3496132914565, 2: -2990.544887377031, 3: -2795.234613920599, 4: -2700.004253051797, 5: -2885.468407004524, 6: -2519.3272041661885, 7: -2467.5831954970417, 8: -2510.214289491267, 9: -1893.7882455848346, 10: -1423.8999509488142, 11: -2033.7317058668746, 12: -1633.3423396949315, 13: -1167.5427029973637, 14: -1082.0996040262362, 15: -1272.2589565380586, 16: -560.6584734282906, 17: -1063.5098314266356, 18: -1665.413920371919, 19: -2057.013760463818, 20: -2202.342498784686, 21: -2274.470515165853, 22: -2296.9609148084196, 23: -2145.1884979025926}
{0: 1715.8213760949704, 1: 1311.510640288112, 2: 1661.8281396462405, 3: 1155.7160722988503, 4: 1060.6702831546636, 5: 840.774155437102, 6: 1338.0571660260357, 7: 1632.5651585068385, 8: 1974.089451212548, 9: 2989.5447100366587, 10: 2454.104006786574, 11: 2831.7028765646087, 12: 2451.185114396613, 13: 2052.788624420927, 14: 2050.619543312498, 15: 2008.7285932754623, 16: 2030.1761328390073, 17: 2627.9725490604756, 18: 2

In [73]:
# Storage means capacity (MW)
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar          138364.610149
dtype: object

In [74]:
Variables['energy_Pvar'].max()

TIME           2019-12-31 23:00:00
energy_Pvar          138364.610149
dtype: object

In [7]:
df=Variables['power_Dvar']
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')]['power_Dvar'].sum())
df2=Variables['storageOut_Pvar']
print(df2[df2.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df3=Variables['storageIn_Pvar']
print(df3[df3.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df4=Variables['stockLevel_Pvar']
print(df4[df4.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(Variables['stockLevel_ini_Dvar'])

                     TIME    TECHNOLOGIES    power_Dvar
40989 2019-12-31 23:00:00     WindOnShore   4432.600000
40990 2019-12-31 23:00:00             CCG      0.000000
40991 2019-12-31 23:00:00         OldNuke  15520.000000
40992 2019-12-31 23:00:00             TAC      0.000000
40993 2019-12-31 23:00:00    WindOffShore  10598.700000
40994 2019-12-31 23:00:00  HydroReservoir   8785.000000
40995 2019-12-31 23:00:00         Biomass   3000.000000
40996 2019-12-31 23:00:00            Coal      0.000000
40997 2019-12-31 23:00:00           Solar      0.000000
40998 2019-12-31 23:00:00         NewNuke  21710.000000
40999 2019-12-31 23:00:00      HydroRiver   8233.891861
41000 2019-12-31 23:00:00   NewHydroRiver    895.799659
41001 2019-12-31 23:00:00     curtailment      0.000000
73175.99151963575
                    TIME      STOCK_TECHNO  storageOut_Pvar
9459 2019-12-31 23:00:00           Battery              0.0
9460 2019-12-31 23:00:00  PowertoH2toPower              0.0
9461 2019-12-31 23

In [8]:
df=Variables['power_Dvar']
df2=Variables['storageOut_Pvar']
df3=Variables['storageIn_Pvar']
df=df.groupby('TIME')['power_Dvar'].sum()
df2=df2.groupby('TIME')['storageOut_Pvar'].sum()
df3=df3.groupby('TIME')['storageIn_Pvar'].sum()
df_prod=pd.DataFrame()
df_prod['Prod']=df+df2-df3
#df_prod
#df_prod=df_prod.set_index('TIME')

df_diff=pd.DataFrame()
df_diff['diff']=df_prod['Prod']-Variables['total_consumption_Pvar'].set_index('TIME')['total_consumption_Pvar']
print(df_diff['diff'].max())
N_err=0
for i in df_diff.index:
    if np.abs(df_diff.loc[i,'diff'])>1:
        N_err+=1
        print(df_diff.loc[i,'diff'])
print(N_err)

4.3655745685100555e-11
0


In [9]:
df_prod.loc[pd.Timestamp('2019-12-31 23:00:00')]

Prod    73175.99152
Name: 2019-12-31 23:00:00, dtype: float64

In [10]:
Variables['total_consumption_Pvar'].set_index('TIME').loc[pd.Timestamp('2019-12-31 23:00:00')]

total_consumption_Pvar    73175.99152
Name: 2019-12-31 23:00:00, dtype: float64

In [21]:
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar            155289.7205
dtype: object

In [12]:
Variables['total_consumption_Pvar'][Variables['total_consumption_Pvar'].total_consumption_Pvar==Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()]

,TIME,total_consumption_Pvar
1290,2019-01-24 13:00:00,150448.942674


In [13]:
d0=pd.Timestamp('2019-01-01 00:00:00')
d1h=timedelta(hours=1)
df=Variables['flex_consumption_Pvar']
L_EV=[]
for h in range(24):
    L_EV.append(float(df[(df.TIME==d0+h*d1h)&(df.FLEX_CONSUM=='EV')]['flex_consumption_Pvar']))
L_EV

[8974.985765378075,
 11252.569669328535,
 10691.645219955992,
 11170.426644899537,
 13510.099229131103,
 15473.200406998334,
 16550.885092623164,
 4425.99954355268,
 5581.337750877695,
 20919.870411678494,
 21837.125896171616,
 23312.874833511825,
 28469.2682218418,
 25998.15287218346,
 27305.36161875611,
 25600.720420410304,
 20896.11392020079,
 18139.698630793904,
 17262.300474917676,
 15669.700657177991,
 15415.259850726845,
 17359.40017950799,
 25017.329324809274,
 23867.510910719506]

In [44]:
def Total_cost(Variables,area):
    return 1/1e9*(Variables['powerCosts_Pvar'][Variables['powerCosts_Pvar'].AREAS==area]['powerCosts_Pvar'].sum()\
            +Variables['capacityCosts_Pvar'][Variables['capacityCosts_Pvar'].AREAS==area]['capacityCosts_Pvar'].sum()\
            +Variables['storageCosts_Pvar'][Variables['storageCosts_Pvar'].AREAS==area]['storageCosts_Pvar'].sum()\
            +Variables['flexCosts_Pvar'][Variables['flexCosts_Pvar'].AREAS==area]['flexCosts_Pvar'].sum())
Total_cost(Variables,'FR')

39.551618023149395